In [ ]:
import ee

ee.Initialize()
from IPython.display import display, Image

In [ ]:
scale = 20

In [ ]:
roi = ee.FeatureCollection("users/nkeikon/myanmar_sr/area2")
s2TNI = ee.Image("users/nkeikon/myanmar_sr/s2image").clip(roi)
classified = ee.Image("users/nkeikon/myanmar_sr/area2_classified")
PA = ee.FeatureCollection("WCMC/WDPA/current/polygons")
palm = ee.FeatureCollection("users/nkeikon/myanmar_sr/palm_area2")
rubber = ee.FeatureCollection("users/nkeikon/myanmar_sr/rubber_area2")
other = ee.FeatureCollection("users/nkeikon/myanmar_sr/other_area2")
bare = ee.FeatureCollection("users/nkeikon/myanmar_sr/bare_area2")
water = ee.FeatureCollection("users/nkeikon/myanmar_sr/water_area2")
shrub = ee.FeatureCollection("users/nkeikon/myanmar_sr/shrub_area2")

In [ ]:
Image(url=classified.randomVisualizer().getThumbUrl({"size": "400"}))

In [ ]:
# Calculate the area by class
names = ["1 oil palm", "2 rubber", "3 other", "4 shrub", "5 bare", "6 water"]
count = classified.eq([1, 2, 3, 4, 5, 6]).rename(names)
total = count.multiply(ee.Image.pixelArea())
areaM2 = total.reduceRegion(ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3)
areaPixel = count.reduceRegion(ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3)

# Create feature collections
featureM2 = ee.Feature(None)
featureM2 = featureM2.set("areaM2", areaM2)
area_M2 = ee.FeatureCollection(featureM2)

featurePixel = ee.Feature(None)
featurePixel = featurePixel.set("areaPixel", areaPixel)
area_Pixel = ee.FeatureCollection(featurePixel)

# Export as csv
export1 = ee.batch.Export.table.toDrive(
    collection=area_M2,
    description="export_area2_m2",
    fileNamePrefix="Kawthaung_area2_m2",
)
export1.start()
export2 = ee.batch.Export.table.toDrive(
    collection=area_Pixel,
    description="export_area2_pixel",
    fileNamePrefix="Kawthaung_area2_pixel",
)
export2.start()

In [ ]:
# query current tasks
tasks = ee.batch.Task.list()
print(tasks[0])
print(tasks[1])
print(tasks[2])